In [24]:
from glob import glob
import pandas as pd
import re
from utils import *

In [26]:
# define test MSF parser
def MSF_parser(df, mapper=mods_dict):
    return df[["Peptide", "Probability", "Assigned Modifications", "Protein"]] 

# define MM parser
def MM_parser(df, df_all, mapper=mods_dict):
    df["MM_score"] = df["Sequence"].map(df_all.set_index("Full Sequence")["Score"])
    df = df[["MM_score", "Base Sequence", "Sequence", "Protein Groups"]]
    return df

IMPORT DATA

In [55]:
# find all files in said folders
msf_filenames = glob('data/msfragger/*')
mm_filenames = glob('data/metamorpheus/*.tsv')
mm_filenames_all = glob('data/metamorpheus/*.psmtsv')

MS FRAGGER

In [31]:
# make dataframes, apply MSF_parser function, save as a list of dataframes
msf_dfs = []
for file in msf_filenames:
    df = pd.read_csv(file, sep='\t')
    df = MSF_parser(df)
    msf_dfs.append(df)

In [32]:
msf_dfs[0]

,Peptide,Probability,Assigned Modifications,Protein
0,AALSDEIEASR,0.9707,NaN,TcCLB.511217.100:mRNA-p1
1,AATLSVPR,0.9773,NaN,TCSYLVIO_005804-t26_1-p1
2,AEENDAGHGATVIPPR,0.9969,NaN,TCSYLVIO_006243-t26_1-p1
3,AGALCLKR,0.9993,5C(57.0214),TcCLB.511145.80:mRNA-p1
4,AGAVSQAYHDLASR,1.0000,NaN,TcCLB.511001.120:mRNA-p1
...,...,...,...,...
181,YLTASALFR,0.9578,NaN,TcCLB.506563.40:mRNA-p1
182,YMACCLMYR,0.9914,"2M(15.9949), 4C(57.0214), 5C(57.0214), 7M(15.9...",TCSYLVIO_008204-t26_1-p1
183,YNPAEVR,0.9415,NaN,TCSYLVIO_007656-t26_1-p1
184,YNVLEVVSK,0.9873,NaN,TcCLB.511139.20:mRNA-p1


same, but different: LIST COMPREHESION

In [35]:
msf_dfs2 = [MSF_parser(pd.read_csv(file, sep='\t')) for file in msf_filenames]

In [36]:
msf_dfs2[0]

,Peptide,Probability,Assigned Modifications,Protein
0,AALSDEIEASR,0.9707,NaN,TcCLB.511217.100:mRNA-p1
1,AATLSVPR,0.9773,NaN,TCSYLVIO_005804-t26_1-p1
2,AEENDAGHGATVIPPR,0.9969,NaN,TCSYLVIO_006243-t26_1-p1
3,AGALCLKR,0.9993,5C(57.0214),TcCLB.511145.80:mRNA-p1
4,AGAVSQAYHDLASR,1.0000,NaN,TcCLB.511001.120:mRNA-p1
...,...,...,...,...
181,YLTASALFR,0.9578,NaN,TcCLB.506563.40:mRNA-p1
182,YMACCLMYR,0.9914,"2M(15.9949), 4C(57.0214), 5C(57.0214), 7M(15.9...",TCSYLVIO_008204-t26_1-p1
183,YNPAEVR,0.9415,NaN,TCSYLVIO_007656-t26_1-p1
184,YNVLEVVSK,0.9873,NaN,TcCLB.511139.20:mRNA-p1


META MOPRHEUS

In [56]:
mm_dfs = []
for file1, file2 in zip(mm_filenames, mm_filenames_all):
    df1 = pd.read_csv(file1, sep='\t')
    df2 = pd.read_csv(file2, sep='\t')
    df = MM_parser(df1, df2)
    mm_dfs.append(df)

In [59]:
mm_dfs[0].head()

,MM_score,Base Sequence,Sequence,Protein Groups
0,14.549,AGAVSQAYHDLASR,AGAVSQAYHDLASR,TcCLB.511001.120:mRNA-p1 | TcCLB.511001.120:mR...
1,11.259,ALVKDEVIPR,ALVKDEVIPR,TcCLB.506743.4:mRNA-p1 | TcCLB.506743.4:mRNA-p...
2,15.500,AVCMIANSTAIAEVFAR,AVC[Common Fixed:Carbamidomethyl on C]M[Common...,TCSYLVIO_008204-t26_1-p1 | TCSYLVIO_008204-t26...
3,11.328,AVLIDLEPGTMDSVR,AVLIDLEPGTM[Common Variable:Oxidation on M]DSVR,TcCLB.506563.40:mRNA-p1 | TcCLB.506563.40:mRNA-p1
4,12.258,CGINYQPPTVVPGGDLAK,C[Common Fixed:Carbamidomethyl on C]GINYQPPTVV...,TCSYLVIO_008204-t26_1-p1 | TCSYLVIO_008204-t26...


In [61]:
# list comprehension
mm_dfs2 = [MM_parser(pd.read_csv(file1, sep='\t'), pd.read_csv(file2, sep='\t')) for file1, file2 in zip(mm_filenames, mm_filenames_all)]

In [62]:
mm_dfs2[0].head()

,MM_score,Base Sequence,Sequence,Protein Groups
0,14.549,AGAVSQAYHDLASR,AGAVSQAYHDLASR,TcCLB.511001.120:mRNA-p1 | TcCLB.511001.120:mR...
1,11.259,ALVKDEVIPR,ALVKDEVIPR,TcCLB.506743.4:mRNA-p1 | TcCLB.506743.4:mRNA-p...
2,15.500,AVCMIANSTAIAEVFAR,AVC[Common Fixed:Carbamidomethyl on C]M[Common...,TCSYLVIO_008204-t26_1-p1 | TCSYLVIO_008204-t26...
3,11.328,AVLIDLEPGTMDSVR,AVLIDLEPGTM[Common Variable:Oxidation on M]DSVR,TcCLB.506563.40:mRNA-p1 | TcCLB.506563.40:mRNA-p1
4,12.258,CGINYQPPTVVPGGDLAK,C[Common Fixed:Carbamidomethyl on C]GINYQPPTVV...,TCSYLVIO_008204-t26_1-p1 | TCSYLVIO_008204-t26...
